## Goal: populate a table with the parsed field values of LabLog

- Implent this using a stored procedure and functions(?)

## Parsing Functions

- \[udf-Str-Parse-Row\]
    
- \[dbo\].\[udf-Str-Parse\]

In [ ]:
CREATE FUNCTION [dbo].[udf-Str-Parse] (@String varchar(max), @Delimiter varchar(10))
Returns Table 
WITH SCHEMABINDING
As
Return (  
    Select RetSeq = Row_Number() over (Order By (Select null))
          , RetVal = LTrim(RTrim(B.i.value('(./text())[1]', 'varchar(max)')))
From (Select x = Cast('<x>' + replace((Select replace(@String,@Delimiter,'§§Split§§') as [*]
        For XML Path('')),'§§Split§§','</x><x>')+'</x>' as xml).query('.')) as A 
    Cross Apply x.nodes('x') AS B(i)
);

GO

In [ ]:
CREATE FUNCTION [dbo].[udf-Str-Parse-Row] (@String varchar(max),@Delimiter varchar(10))
Returns Table 
WITH SCHEMABINDING
As
Return (
    Select 
          Pos1 = ltrim(rtrim(xDim.value('/x[1]','varchar(255)')))
        , Pos2 = ltrim(rtrim(xDim.value('/x[2]','varchar(255)')))
        , Pos3 = ltrim(rtrim(xDim.value('/x[3]','varchar(255)')))
        , Pos4 = ltrim(rtrim(xDim.value('/x[4]','varchar(255)')))
        , Pos5 = ltrim(rtrim(xDim.value('/x[5]','varchar(255)')))
        , Pos6 = ltrim(rtrim(xDim.value('/x[6]','varchar(255)')))
        , Pos7 = ltrim(rtrim(xDim.value('/x[7]','varchar(255)')))
        , Pos8 = ltrim(rtrim(xDim.value('/x[8]','varchar(255)')))
        , Pos9 = ltrim(rtrim(xDim.value('/x[9]','varchar(255)')))
        , Pos10 = ltrim(rtrim(xDim.value('/x[10]','varchar(255)')))
        , Pos11 = ltrim(rtrim(xDim.value('/x[11]','varchar(255)')))
        , Pos12 = ltrim(rtrim(xDim.value('/x[12]','varchar(255)')))
        , Pos13 = ltrim(rtrim(xDim.value('/x[13]','varchar(255)')))
    From  (Select Cast('<x>' + replace((Select replace(@String,@Delimiter,'§§Split§§') as [*] For XML Path('')),'§§Split§§','</x><x>')+'</x>' as xml) as xDim) as A 
)

GO

## Example Usage

- Create a view of parsed row data

### Example of a message

<span style="color: #657b83;font-style: italic;">Col Content</span>

<span style="color: #657b83;font-style: italic;">1&nbsp; Nov&nbsp;19&nbsp;11:28:17&nbsp;DC2.stern.com&nbsp;MSWinEventLog&nbsp;&nbsp;&nbsp;&nbsp;</span> 

<span style="color: #657b83;font-style: italic;">2&nbsp; 5&nbsp;&nbsp;</span> 

<span style="color: #657b83;font-style: italic;">3&nbsp;</span> <span style="color: rgb(101, 123, 131); font-style: italic;">Security&nbsp;&nbsp;&nbsp;</span> 

<span style="color: #657b83;font-style: italic;">4&nbsp;</span> <span style="color: rgb(101, 123, 131); font-style: italic;">22418&nbsp;&nbsp;</span> 

<span style="color: #657b83;font-style: italic;">5&nbsp;</span> <span style="color: rgb(101, 123, 131); font-style: italic;">Thu&nbsp;Nov&nbsp;19&nbsp;11:28:10&nbsp;2020&nbsp;&nbsp;&nbsp;</span> 

<span style="color: #657b83;font-style: italic;">6&nbsp;</span> <span style="color: rgb(101, 123, 131); font-style: italic;">4634&nbsp;&nbsp;&nbsp;</span> 

<span style="color: #657b83;font-style: italic;">7&nbsp;</span> <span style="color: rgb(101, 123, 131); font-style: italic;">Microsoft-Windows-Security-Auditing&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span> 

<span style="color: #657b83;font-style: italic;">8&nbsp;</span> <span style="color: rgb(101, 123, 131); font-style: italic;">N/A&nbsp;&nbsp;&nbsp;&nbsp;</span> 

<span style="color: #657b83;font-style: italic;">9&nbsp;</span> <span style="color: rgb(101, 123, 131); font-style: italic;">Audit&nbsp;Success&nbsp;&nbsp;</span> 

<span style="color: #657b83;font-style: italic;">10</span> <span style="color: rgb(101, 123, 131); font-style: italic;">&nbsp;DC2.stern.com&nbsp;</span> 

<span style="color: #657b83;font-style: italic;">11</span> <span style="color: rgb(101, 123, 131); font-style: italic;">&nbsp;12545&nbsp;</span> 

<span style="color: #657b83;font-style: italic;">12</span> <span style="color: rgb(101, 123, 131); font-style: italic;">&nbsp;An&nbsp;account&nbsp;was&nbsp;logged&nbsp;off.Subject:&nbsp;&nbsp;&nbsp;&nbsp;</span>

In [ ]:
USE LabLog;
GO

CREATE VIEW dbo.ParseFields
AS

Select TOP 100
    A.Id
    -- ,B.RetSeq
    -- ,B.RetVal
    -- , B.*
    -- , C.Pos6 AS 'Level'
    , C.Pos3 AS 'Category'
    , C.Pos5 AS 'TimeStamp'
    , C.Pos6 AS 'EventID'
    , C.Pos7 AS 'Windows Log'
    , C.Pos10 AS 'Short Message'
    , C.Pos11 AS 'Host Name'
    , C.Pos13 AS 'Message Subject'
From dbo.Logs1 A
 Cross Apply [dbo].[udf-Str-Parse](A.Message,char(13)) B
 Cross Apply [dbo].[udf-Str-Parse-Row](B.RetVal,char(9)) C
Where B.RetVal is not null and B.RetSeq = 1

GO